## Plot data

In [ ]:
from pathlib import Path
import warnings

import h5py
import numpy as np
import scipy as sp
from scipy.optimize import curve_fit
import skimage as ski
import matplotlib.pyplot as plt
plt.rcParams['text.usetex'] = True
plt.rcParams.update({'font.size': 12})
import os

from fusion import optimize
from utils import hyperspectral2RGB, hyperspectral2RGBvolume, bin_data
from utils import RESOURCES_PATH
from ipywidgets import interact

RESOURCES_PATH = Path("/Users/federicosimoni/Library/Mobile Documents/com~apple~CloudDocs/Università/Tesi/Code/CS-FLIM_lab/20240703/kidney")

DATA_PATH = RESOURCES_PATH / "520_kidney2_550_550_610_SPC_raw_proc_tlxy_256x256.npz"

In [ ]:
data = np.load(DATA_PATH)
x = data['x']
t = data['t']
lam = data['lam']

XY_DIM = x.shape[-1]
zoom_factor = 8
x = x[:, :, :, XY_DIM // 2 - XY_DIM // zoom_factor:XY_DIM // 2 + XY_DIM // zoom_factor,
            XY_DIM // 2 - XY_DIM // zoom_factor:XY_DIM // 2 + XY_DIM // zoom_factor]

slices_rgb = hyperspectral2RGBvolume(lam, np.mean(x, axis=0))

means_spectrums = np.mean(x, axis=(0, 3, 4)).T
min_mean_spectrum = np.min(means_spectrums)
max_mean_spectrum = np.max(means_spectrums)

means_times = np.mean(x, axis=(1, 3, 4)).T
min_mean_times = np.min(means_times)
max_mean_times = np.max(means_times)

zxy = np.sum(x, axis=(0, 1))
zxy /= zxy.max()

def plot_across_z(index_z=0):
    _, ax = plt.subplots(2, 2, figsize=(8, 8))
    ax[0, 0].imshow(zxy[index_z], cmap="gray", vmin=0, vmax=1)
    ax[0, 0].set_title(f"Fused image, z={index_z}")

    ax[0, 1].imshow(slices_rgb[index_z])
    ax[0, 1].set_title(f"Colored with spectrum")

    ax[1, 0].plot(lam, means_spectrums[index_z])
    ax[1, 0].set_ylim(min_mean_spectrum, max_mean_spectrum)
    ax[1, 0].set_title(f"Fused Spectrum, z={index_z}")
    ax[1, 0].grid()

    ax[1, 1].plot(t, means_times[index_z])
    ax[1, 1].set_ylim(min_mean_times, max_mean_times)
    ax[1, 1].set_title(f"Fused Time, z={index_z} - {len(t)} temporal points")
    ax[1, 1].grid()

    plt.tight_layout()
    plt.savefig('prova.pdf', format='pdf')
    plt.show()


interact(plot_across_z, index_z=(0, len(zxy) - 1, 1));

In [ ]:
z_index = np.argmax(means_spectrums.mean(axis=1))

reconstructed_spectrums = np.mean(x[:,:,z_index,:,:], axis=(0))
min_spectrums, max_spectrums = np.min(reconstructed_spectrums), np.max(reconstructed_spectrums)

reconstructed_times = np.mean(x[:,:,z_index,:,:], axis=(1))
min_times, max_times = np.min(reconstructed_times), np.max(reconstructed_times)

def reconstructed_spectrum_time_in_a_point(i=0, j=0):
    _, ax = plt.subplots(1, 3, figsize=(12, 4))
    ax[0].imshow(slices_rgb[z_index])
    ax[0].scatter([j], [i], c="b")
    ax[0].set_title(f"Colored with spectrum")
    
    ax[1].plot(lam, reconstructed_spectrums[:, i, j])
    ax[1].set_title(f"Spectrum in ({i},{j})")
    ax[1].set_ylim(min_spectrums, max_spectrums)
    ax[1].grid()
    
    ax[2].plot(t, reconstructed_times[:, i, j])
    ax[2].set_title(f"Time in ({i},{j})")
    ax[2].set_ylim(min_times, max_times)
    ax[2].grid()
    
    plt.tight_layout()
    plt.show()
    
interact(reconstructed_spectrum_time_in_a_point, i=(0, x.shape[-2], 1), j=(0, x.shape[-1], 1));

## Convert as Matlab file

In [ ]:
# save = True
# if (save):
#     mdic = {"x": x, "t":t, "lam":lam}
#     SAVE_PATH = RESOURCES_PATH / "520_kidney2_550_550_610_SPC_raw_proc_tlxy_fused_64x64_512times.mat"
#     sp.io.savemat(SAVE_PATH, mdic)

## Time plot

In [ ]:
def func(x, a, b, c):
    return a * np.exp(-b * x) + c

time_decay = np.mean(x, axis=(1,2))[:,15,20]
plt.plot(t, time_decay)

popt, pcov = curve_fit(func, t, time_decay)
fit1 = popt[0] * np.exp(-popt[1]*t) + popt[2]
# fit2 = popt[3] * np.exp(-popt[4]*t) + popt[2]

plt.plot(t, fit1, label='Fitted Function:\n $y = %0.2f e^{-t/%0.2f} + %0.2f$' % (popt[0], 1/popt[1], popt[2]))
# plt.plot(t, fit2, label='Fitted Function:\n $y = %0.2f e^{-t/%0.2f} + %0.2f$' % (popt[3], 1/popt[4], popt[2]))
plt.legend()
plt.show()

In [ ]:
time_plot = np.zeros((x.shape[3],x.shape[4]))
time_decay_matrix = np.mean(x, axis=(1,2)).astype(np.float64)

for i in range(x.shape[3]):
    for j in range(x.shape[4]):
        time_decay = time_decay_matrix[:,i,j]
        popt, pcov = curve_fit(func, t, time_decay, maxfev=500000)
        time_plot[i,j] = 1/popt[1]
        
plt.imshow(time_plot, vmin=0, vmax=4.5)
plt.colorbar()
plt.savefig('prova.pdf', format='pdf')
plt.show()

In [ ]:
print(time_plot[22,6])

In [ ]:
SAVE_PATH = RESOURCES_PATH / "520_kidney2_550_550_610_SPC_raw_proc_tlxy_fused_64x64_512times_plots.npz"
if not SAVE_PATH.is_file():
    np.savez(SAVE_PATH, time_plot=time_plot)
else:
    warnings.warn("File name already exists!")